# Dask DataFrame

 - Dask is a flexible library for parallel computing using Python.
 - Besides other powerful tools, we'll focus on Dask DataFrame. 
 - It reuses a lot of pandas code but it extends it to a larger scale. 
 - A Dask DataFrame is a large parallel DataFrame composed of many smaller Pandas DataFrames, split along an index


In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
Image(url="http://dask.pydata.org/en/latest/_images/dask-dataframe.svg",width=400)

### Reading the data - National Food Survey UK 

The National Food Survey (NFS), which closed in 2000, was the longest-running continuous survey of household food consumption and expenditure in the world. It was originally set up in 1940 by the then Ministry of Food to monitor the adequacy of the diet of urban 'working class' households in wartime, but it was extended in 1950 to become representative of households throughout Great Britain (the UKDA holds NFS data from 1974-2000 only). 
- Information: https://www.gov.uk/government/statistics/family-food-open-data
- Source: http://britains-diet.labs.theodi.org/data/

We have a zip file that contains all the .csv files that we'll read. 

In [3]:
import zipfile

zf = zipfile.ZipFile('Data/nfs.zip')
zf.extractall(path='Data/')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('Data/nfs/NFS_2000.csv')
df

,hhno,fditemno,logday,pv,minfd,cq,pf,styr,stmth,memhh,adltm,adltf,child,oaps,adltgt64
0,261119,27,6,36,31901,6.77,1,2000,1,1,1,0,0,0,0
1,261119,28,6,49,26001,28.21,1,2000,1,1,1,0,0,0,0
2,261119,29,7,58,1503,1.75,1,2000,1,1,1,0,0,0,0
3,261119,30,7,49,26001,28.21,1,2000,1,1,1,0,0,0,0
4,261119,31,7,250,7301,32.00,1,2000,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237419,267819,20,4,149,15503,176.36,1,2000,12,4,1,1,2,0,0
237420,267819,21,4,36,404,1.00,1,2000,12,4,1,1,2,0,0
237421,267819,22,6,72,404,2.00,1,2000,12,4,1,1,2,0,0
237422,267819,23,6,59,25701,28.21,1,2000,12,4,1,1,2,0,0


 ### Some variables from the dataset
 - hhno : household id
 - styr : year column
 - minfd : identifier of food category
 - cq: consumption
 - memhh : number of members of the household

As usual, we can do pandas operations... 

In [6]:
df.groupby('minfd').minfd.count()

minfd
402        48
403       272
404      7065
501       130
601      1148
         ... 
38501      89
38601     162
38701     384
38801      43
38901      90
Name: minfd, Length: 242, dtype: int64

In [7]:
df.groupby('minfd').cq.mean()

minfd
402         4.890625
403         2.620147
404         2.684357
501         2.416846
601         0.402892
            ...     
38501      99.323596
38601      91.135802
38701      75.584167
38801      67.465116
38901    1133.055556
Name: cq, Length: 242, dtype: float64

## What can we do with Dask ?

Most of the functions are part of pandas dataframe API: https://docs.dask.org/en/latest/dataframe-api.html

Some examples of parallelizable operations (fast):

- Elementwise operations: `df.x + df.y, df * df`
- Row-wise selections: `df[df.x > 0]`
- Loc: `df.loc[4.0:10.5]`
- Common aggregations: `df.x.max(), df.max()`
- Is in: `df[df.x.isin([1, 2, 3])]`


In [8]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:50104 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [9]:
import dask
import dask.dataframe as dd

In [10]:
# Notice that we are reading ALL csv files. 
df = dd.read_csv("Data/nfs/NFS*.csv")

Notice that:
- Nothing yet is loaded in to memory
- Meta-information from pandas is available

In [11]:
df

,hhno,fditemno,logday,pv,minfd,cq,pf,styr,stmth,memhh,adltm,adltf,child,oaps,adltgt64
npartitions=27,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
# DataFrame.head is one operation that is not lazy
df.head(5)

,hhno,fditemno,logday,pv,minfd,cq,pf,styr,stmth,memhh,adltm,adltf,child,oaps,adltgt64
0,20001,1,1,6,401,1.0,1,1974,1,4,1,1,2,0,0
1,20001,2,1,11,25201,28.0,1,1974,1,4,1,1,2,0,0
2,20001,3,1,50,12901,12.0,1,1974,1,4,1,1,2,0,0
3,20001,4,1,26,24001,12.0,1,1974,1,4,1,1,2,0,0
4,20001,5,1,11,30401,4.0,1,1974,1,4,1,1,2,0,0


- styr : year column
- minfd : identifier of food category

In [13]:
df.cq.mean()

dd.Scalar<series-..., dtype=float64>

In [14]:
df.cq.mean().compute()

18.903555021993032

###  Partitions

- By default the data is partitioned by each file
- In our case, this is good. The files have a natural partition


In [15]:
df.npartitions
# Number of. CSV files

27

#### Each partition is just a pandas DataFrame

In [16]:
df.partitions[5].compute()

,hhno,fditemno,logday,pv,minfd,cq,pf,styr,stmth,memhh,adltm,adltf,child,oaps,adltgt64
0,70001,1,1,14,401,1.0,1,1979,1,2,1,1,0,0,0
1,70001,2,1,85,3104,15.0,1,1979,1,2,1,1,0,0,0
2,70001,3,1,76,7301,17.0,1,1979,1,2,1,1,0,0,0
3,70001,4,1,35,20301,16.0,1,1979,1,2,1,1,0,0,0
4,70001,5,1,60,5503,8.0,1,1979,1,2,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334307,77277,37,7,26,22801,16.0,1,1979,12,2,1,1,0,2,2
334308,77277,38,7,38,17801,10.0,1,1979,12,2,1,1,0,2,2
334309,77277,39,7,20,16701,7.0,1,1979,12,2,1,1,0,2,2
334310,77277,40,7,28,16301,32.0,1,1979,12,2,1,1,0,2,2


In [17]:
type(df.partitions[5].compute())

pandas.core.frame.DataFrame

In [18]:
df.known_divisions

False

In [19]:
df = df.set_index('styr', sorted=True)

In [20]:
df.known_divisions

True

In [21]:
df.head()

,hhno,fditemno,logday,pv,minfd,cq,pf,stmth,memhh,adltm,adltf,child,oaps,adltgt64
styr,,,,,,,,,,,,,,
1974,20001,1,1,6,401,1.0,1,1,4,1,1,2,0,0
1974,20001,2,1,11,25201,28.0,1,1,4,1,1,2,0,0
1974,20001,3,1,50,12901,12.0,1,1,4,1,1,2,0,0
1974,20001,4,1,26,24001,12.0,1,1,4,1,1,2,0,0
1974,20001,5,1,11,30401,4.0,1,1,4,1,1,2,0,0


In [22]:
df.info()


<class 'dask.dataframe.core.DataFrame'>
Columns: 14 entries, hhno to adltgt64
dtypes: float64(1), int64(13)

In [23]:
df.divisions

(1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2000)

## Fast access to subsets of data

In [24]:
df.loc[2000]

,hhno,fditemno,logday,pv,minfd,cq,pf,stmth,memhh,adltm,adltf,child,oaps,adltgt64
npartitions=1,,,,,,,,,,,,,,
2000,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64
2000,...,...,...,...,...,...,...,...,...,...,...,...,...,...


However, this IS a lazy operation. So, we need to add `compute()`.

In [25]:
df.loc[2000].compute()

,hhno,fditemno,logday,pv,minfd,cq,pf,stmth,memhh,adltm,adltf,child,oaps,adltgt64
styr,,,,,,,,,,,,,,
2000,261119,27,6,36,31901,6.77,1,1,1,1,0,0,0,0
2000,261119,28,6,49,26001,28.21,1,1,1,1,0,0,0,0
2000,261119,29,7,58,1503,1.75,1,1,1,1,0,0,0,0
2000,261119,30,7,49,26001,28.21,1,1,1,1,0,0,0,0
2000,261119,31,7,250,7301,32.00,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000,267819,20,4,149,15503,176.36,1,12,4,1,1,2,0,0
2000,267819,21,4,36,404,1.00,1,12,4,1,1,2,0,0
2000,267819,22,6,72,404,2.00,1,12,4,1,1,2,0,0


In [26]:
len(df)

8338857

In [27]:
df.groupby('minfd').minfd.count().nlargest(10).compute()  # aggregation API is fast

minfd
401      464128
404      303117
27001    165568
12901    164029
1503     158015
21701    157175
25201    148183
17801    147401
27401    142734
17501    113490
Name: minfd, dtype: int64

In [28]:
minfd = 401

In [29]:
import pandas as pd
food_mapping = pd.read_csv("Data/food_mapping.csv", index_col='minfd')

In [30]:
food_mapping

,minfddesc,majfd,majfddesc,Coded unit,fdgp,fd gp description
minfd,,,,,,
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4005.0,"LIQUID WHOLEMILK, FULL PRICE & WELFARE"
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4006.0,"LIQUID WHOLEMILK, INC SCHOOL & WELFARE"
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4017.0,TOTAL MILK AND CREAM
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4023.0,"TOTAL MILK, CREAM AND CHEESE"
402,"UHT LIQUID MILK, FULL PRICE",4,"LIQUID WHOLEMILK, FULL PRICE",pt,4005.0,"LIQUID WHOLEMILK, FULL PRICE & WELFARE"
...,...,...,...,...,...,...
38701,SPIRITS,387,SPIRITS,cl,384388.0,"WINES, SPIRITS AND LIQUEURS"
38801,LIQUEURS AND COCKTAILS,388,LIQUEURS,cl,380389.0,ALCOHOLIC DRINKS
38801,LIQUEURS AND COCKTAILS,388,LIQUEURS,cl,384388.0,"WINES, SPIRITS AND LIQUEURS"


#### What was the most consumed food group in 1974? 
#### What was the most consumed food group overall?

In [31]:
df_1974 = df.loc[1974]

In [32]:
top_74 = df_1974.minfd.value_counts().nlargest(10).compute()

In [33]:
top_74

401      50031
25201    12374
12901     9406
27001     8540
5501      8082
27402     7807
21701     7136
15001     7000
17801     6479
30401     5709
Name: minfd, dtype: int64

In [34]:
food_mapping.loc[401]
#food_mapping.loc[25201]
#food_mapping.loc[12901]

,minfddesc,majfd,majfddesc,Coded unit,fdgp,fd gp description
minfd,,,,,,
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4005.0,"LIQUID WHOLEMILK, FULL PRICE & WELFARE"
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4006.0,"LIQUID WHOLEMILK, INC SCHOOL & WELFARE"
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4017.0,TOTAL MILK AND CREAM
401,MILK LIQUID FULL PRICE,4,"LIQUID WHOLEMILK, FULL PRICE",pt,4023.0,"TOTAL MILK, CREAM AND CHEESE"


In [35]:
top = df.minfd.value_counts().nlargest(10).compute()

In [36]:
top

401      464128
404      303117
27001    165568
12901    164029
1503     158015
21701    157175
25201    148183
17801    147401
27401    142734
17501    113490
Name: minfd, dtype: int64

### map_partitions
- If we have a code that works well on a single data frame and we want to apply an "embarransingly parallel way" across many pandas data frames that live inside the `dask` data frame. 
- Map partitions does what you might expect
- Maps a function across partitions

In [37]:
df.map_partitions(len).compute()

0     369072
1     361147
2     362535
3     370360
4     356588
5     334312
6     358177
7     344527
8     350645
9     314468
10    300210
11    298411
12    294980
13    301459
14    298952
15    310683
16    274951
17    259880
18    293403
19    307705
20    295011
21    303074
22    322933
23    247462
24    236079
25    234409
26    237424
dtype: int64

### Let's calculate the most frequently purchase food group for each year

In [38]:
def most_frequent_food(partition):
    # partition is a pandas.DataFrame
    minfd = partition.minfd.value_counts().nlargest(1).index[0]
    description = food_mapping.loc[minfd].minfddesc.iloc[0]
    year = int(partition.index[0])
    return pd.DataFrame({'year': [year], 'description': [description]})

In [39]:
mnfd_year = df.map_partitions(most_frequent_food, 
                              meta={'year': int,
                                    'description': str})

In [40]:
mnfd_year.compute()


,year,description
0,1974,MILK LIQUID FULL PRICE
0,1975,MILK LIQUID FULL PRICE
0,1976,MILK LIQUID FULL PRICE
0,1977,MILK LIQUID FULL PRICE
0,1978,MILK LIQUID FULL PRICE
0,1979,MILK LIQUID FULL PRICE
0,1980,MILK LIQUID FULL PRICE
0,1981,MILK LIQUID FULL PRICE
0,1982,MILK LIQUID FULL PRICE
0,1983,MILK LIQUID FULL PRICE
